# 05 - Dictionaries (Teams & Players)
Génération et aperçu des dictionnaires équipes/joueurs.


In [1]:
import sys
from pathlib import Path

def _find_root():
    cand = Path.cwd()
    for c in [cand, *cand.parents]:
        if (c / 'src').exists() and (c / 'data').exists():
            return c
    return cand

ROOT = _find_root()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

processed_dir = ROOT / 'data/processed'
parquet_paths = [processed_dir / f for f in ['matches.parquet', 'players.parquet', 'objectives.parquet', 'teamfights.parquet']]
missing = [p for p in parquet_paths if not p.exists()]
if missing:
    raise FileNotFoundError(f"Parquets manquants: {missing}. Regenerer avec `make parquet`.")

import polars as pl
from src.dota_data import read_processed_tables, build_team_dictionary, build_player_dictionary

tables = read_processed_tables(processed_dir)
teams = build_team_dictionary(tables['matches'])
players = build_player_dictionary(tables['players'])

out_dir = ROOT / 'data/dictionaries'
out_dir.mkdir(parents=True, exist_ok=True)
teams_path = out_dir / 'teams.parquet'
players_path = out_dir / 'players.parquet'
teams.write_parquet(teams_path)
players.write_parquet(players_path)

teams.shape, players.shape


((180, 5), (637, 4))

In [5]:
teams.head(10)


team_id,name,tag,logo_url,side_sampled
i64,str,str,str,str
9530783,"""Team Hryvnia""","""TH""","""https://cdn.steamusercontent.c…","""radiant"""
9600141,"""Zero Tenacity""","""Z10""","""https://cdn.steamusercontent.c…","""radiant"""
9861642,"""BanyaPivoSamogon""","""BPS""","""https://cdn.steamusercontent.c…","""dire"""
9716756,"""END GAME""","""END""","""https://cdn.steamusercontent.c…","""dire"""
9651814,"""BanyaPivoSamogon""","""""","""https://cdn.steamusercontent.c…","""radiant"""
9814008,"""eSpoiled""","""eSpoiled""","""https://cdn.steamusercontent.c…","""radiant"""
9885130,"""Perú Rejects""","""PERU""","""https://cdn.steamusercontent.c…","""dire"""
9572001,"""PARIVISION""","""PV""","""https://cdn.steamusercontent.c…","""dire"""
9895392,"""Virtus.pro""","""VP""","""https://cdn.steamusercontent.c…","""radiant"""


In [3]:
players.head(10)


account_id,matches_played,personaname,name
i64,u32,str,str
206097366,337,"""99.9suicide""","""queezy"""
185059559,335,"""-""","""Till The End"""
363739653,335,"""поганка""","""alberkaaa"""
93618577,332,"""BRUCE-""","""bzm"""
294135421,325,"""просто балуюсь""","""Nicky`Cool"""
103735745,321,"""lostreak chicken""","""Saksa"""
86698277,321,"""not human (животное)""","""33"""
190826739,311,"""Over My Dead Body""","""BOOM"""
108958769,309,"""sustain""","""panto"""
